<a href="https://colab.research.google.com/github/akash-cs13/MSE_Sem2/blob/main/classification_modeltraining1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import zipfile

In [ ]:
!unzip -o data.zip -d data

In [4]:

!pip install torch torchvision

# Import Libraries
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import copy


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [6]:

data_transforms = {
    'train': transforms.Compose([
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    'val': transforms.Compose([
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [7]:

def create_datasets(data_dir, transform, val_split=0.2):
    full_dataset = datasets.ImageFolder(data_dir, transform)
    dataset_size = len(full_dataset)
    val_size = int(dataset_size * val_split)
    train_size = dataset_size - val_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
    return train_dataset, val_dataset


In [9]:
# Create datasets
data_dir = 'data'
train_dataset, val_dataset = create_datasets(data_dir, data_transforms['train'], val_split=0.2)


In [10]:
# Create dataloaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=4)
}


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [11]:
dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset)
}

class_names = train_dataset.dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [15]:
# Define the Model
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model



In [16]:
# Train ResNet18
resnet18 = models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, len(class_names))
resnet18 = resnet18.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_resnet = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_resnet, step_size=7, gamma=0.1)

resnet18 = train_model(resnet18, criterion, optimizer_resnet, exp_lr_scheduler, num_epochs=10)



Epoch 1/10
----------
train Loss: 0.6512 Acc: 0.6700
val Loss: 0.4866 Acc: 0.8267

Epoch 2/10
----------
train Loss: 0.3784 Acc: 0.8812
val Loss: 0.2581 Acc: 0.9333

Epoch 3/10
----------
train Loss: 0.2264 Acc: 0.9274
val Loss: 0.1760 Acc: 0.9333

Epoch 4/10
----------
train Loss: 0.1410 Acc: 0.9505
val Loss: 0.1542 Acc: 0.9600

Epoch 5/10
----------
train Loss: 0.1233 Acc: 0.9472
val Loss: 0.0954 Acc: 0.9733

Epoch 6/10
----------
train Loss: 0.1228 Acc: 0.9571
val Loss: 0.0976 Acc: 0.9600

Epoch 7/10
----------
train Loss: 0.0640 Acc: 0.9835
val Loss: 0.0861 Acc: 0.9733

Epoch 8/10
----------
train Loss: 0.0660 Acc: 0.9835
val Loss: 0.0546 Acc: 1.0000

Epoch 9/10
----------
train Loss: 0.0600 Acc: 0.9868
val Loss: 0.0767 Acc: 0.9867

Epoch 10/10
----------
train Loss: 0.0581 Acc: 0.9868
val Loss: 0.0693 Acc: 1.0000

Best val Acc: 1.000000


In [17]:
# Train MobileNetV3
mobilenet_v3 = models.mobilenet_v3_large(pretrained=True)
num_ftrs = mobilenet_v3.classifier[3].in_features
mobilenet_v3.classifier[3] = nn.Linear(num_ftrs, len(class_names))
mobilenet_v3 = mobilenet_v3.to(device)

optimizer_mobilenet = optim.SGD(mobilenet_v3.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_mobilenet, step_size=7, gamma=0.1)

mobilenet_v3 = train_model(mobilenet_v3, criterion, optimizer_mobilenet, exp_lr_scheduler, num_epochs=10)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 115MB/s] 


Epoch 1/10
----------
train Loss: 0.6579 Acc: 0.6073
val Loss: 0.7278 Acc: 0.4800

Epoch 2/10
----------
train Loss: 0.5492 Acc: 0.7921
val Loss: 0.6051 Acc: 0.7600

Epoch 3/10
----------
train Loss: 0.4426 Acc: 0.8614
val Loss: 0.4936 Acc: 0.8400

Epoch 4/10
----------
train Loss: 0.3671 Acc: 0.8845
val Loss: 0.4190 Acc: 0.9200

Epoch 5/10
----------
train Loss: 0.2864 Acc: 0.9241
val Loss: 0.3624 Acc: 0.9333

Epoch 6/10
----------
train Loss: 0.2620 Acc: 0.9340
val Loss: 0.2563 Acc: 0.9333

Epoch 7/10
----------
train Loss: 0.2045 Acc: 0.9505
val Loss: 0.2347 Acc: 0.9200

Epoch 8/10
----------
train Loss: 0.2014 Acc: 0.9373
val Loss: 0.2384 Acc: 0.9067

Epoch 9/10
----------
train Loss: 0.2064 Acc: 0.9340
val Loss: 0.2513 Acc: 0.8933

Epoch 10/10
----------
train Loss: 0.1887 Acc: 0.9604
val Loss: 0.2126 Acc: 0.9467

Best val Acc: 0.946667


In [18]:
# Save the Models
torch.save(resnet18.state_dict(), 'my_flower_resnet18.pth')
torch.save(mobilenet_v3.state_dict(), 'my_flower_mobilenet_v3.pth')
